# Download and Clean Dataset

In [1]:
import pandas as pd
import numpy as np

## Let's download the data and read it into a pandas dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


## Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

## Let's check the dataset for any missing values.

In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Split data into predictors and target

In [5]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

## Normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [8]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

## import the libraries

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import keras

from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


# Part A Build a baseline model

## Randomly split the data into a training and test sets by holding 30% of the data for testing.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [11]:
#build the regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train and test the model

In [12]:
model = regression_model()

Instructions for updating:
Colocations handled automatically by placer.


## Train the model with 50 epoachs

In [13]:
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
721/721 [==============================] - 1s 882us/step - loss: 1582.4168
Epoch 2/50
721/721 [==============================] - 0s 141us/step - loss: 1565.0811
Epoch 3/50
721/721 [==============================] - 0s 159us/step - loss: 1547.5283
Epoch 4/50
721/721 [==============================] - 0s 140us/step - loss: 1529.5367
Epoch 5/50
721/721 [==============================] - 0s 140us/step - loss: 1510.9121
Epoch 6/50
721/721 [==============================] - 0s 138us/step - loss: 1491.6362
Epoch 7/50
721/721 [==============================] - 0s 138us/step - loss: 1471.67980s - loss: 1421.71
Epoch 8/50
721/721 [==============================] - 0s 138us/step - loss: 1451.0557
Epoch 9/50
721/721 [==============================] - 0s 140us/step - loss: 1429.5935
Epoch 10/50
721/721 [==============================] - 0s 146us/step - loss: 1406.8245
Epoch 11/50
721/721 [==============================] - 0s 136us/step - lo

## Evaluate the model using test data

In [14]:

loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val


309/309 [==============================] - 0s 197us/step


338.6520102294132

In [15]:
#  compute the mean squared error between the predicted concrete strength and the actual concrete strength
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

338.6520076223858 0.0


In [16]:
#Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors
total_mse = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mse):
    #X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mse) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 153.3060706178733
MSE 2: 109.01760676609274
MSE 3: 86.14635595920402
MSE 4: 75.66015039832847
MSE 5: 68.87094506322373
MSE 6: 62.82472751210037
MSE 7: 54.630285874153806
MSE 8: 50.84225745339995
MSE 9: 48.275496930366195
MSE 10: 46.32323411367472
MSE 11: 45.499464510328174
MSE 12: 44.003234474404344
MSE 13: 43.944041674962705
MSE 14: 43.05254039332319
MSE 15: 42.91632231308033
MSE 16: 42.54754368927101
MSE 17: 42.76141208661027
MSE 18: 42.08386765637444
MSE 19: 42.342365487107955
MSE 20: 42.35723120803586
MSE 21: 41.97036896245765
MSE 22: 41.9675872333613
MSE 23: 41.69700328123222
MSE 24: 41.734957667230404
MSE 25: 41.42422635346941
MSE 26: 41.463737234714344
MSE 27: 41.089993887348854
MSE 28: 41.245618888089574
MSE 29: 41.20905060752696
MSE 30: 40.555208237040006
MSE 31: 40.617398104621365
MSE 32: 40.06329110525187
MSE 33: 40.02036203304155
MSE 34: 39.581082760709
MSE 35: 39.64417863123625
MSE 36: 39.427970978820206
MSE 37: 39.594802844100016
MSE 38: 39.18075176350121
MSE 39: 3

# Compared with partA, when normalizing the data, the mean is smaller and the standard deviation is bigger